In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0-preview is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import tensorflow as tf
from tensorflow import keras

In [2]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [3]:
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode="fan_avg",
                                                distribution="uniform")
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

In [4]:
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
layer = keras.layers.Dense(10, activation=leaky_relu,
                          kernel_initializer="he_normal")

In [5]:
layer = keras.layers.Dense(10, activation="selu",
                          kernel_initializer="lecun_normal")

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),    
    keras.layers.Dense(500, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

W0830 21:48:39.812587 139662994405184 deprecation.py:323] From /home/joschi/anaconda3/envs/tfgpu/lib/python3.7/site-packages/tensorflow/python/keras/backend.py:4149: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_4 (Dense)              (None, 500)               392500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 500)               2000      
_________________________________________________________________
dense_5 (Dense)              (None, 300)               150300    
_________________________________________________________________
batch_normalization_2 (Batch (None, 300)               1200      
_________________________________________________________________
dense_6 (Dense)              (None, 100)               3

In [8]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [9]:
model.layers[1].updates

[<tf.Operation 'cond/Identity' type=Identity>,
 <tf.Operation 'cond_1/Identity' type=Identity>]

In [10]:
optimizer = keras.optimizers.Adam()
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=optimizer,
              metrics=["accuracy"])

In [11]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [12]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                 restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 10s 182us/sample - loss: 0.4842 - accuracy: 0.8269 - val_loss: 0.3485 - val_accuracy: 0.8734
Epoch 2/100
55000/55000 [==============================] - 9s 165us/sample - loss: 0.3759 - accuracy: 0.8621 - val_loss: 0.3334 - val_accuracy: 0.8758
Epoch 3/100
55000/55000 [==============================] - 9s 167us/sample - loss: 0.3345 - accuracy: 0.8771 - val_loss: 0.3192 - val_accuracy: 0.8798
Epoch 4/100
55000/55000 [==============================] - 9s 163us/sample - loss: 0.3074 - accuracy: 0.8857 - val_loss: 0.3095 - val_accuracy: 0.8868
Epoch 5/100
55000/55000 [==============================] - 9s 161us/sample - loss: 0.2852 - accuracy: 0.8943 - val_loss: 0.2966 - val_accuracy: 0.8886
Epoch 6/100
55000/55000 [==============================] - 9s 163us/sample - loss: 0.2651 - accuracy: 0.8992 - val_loss: 0.3072 - val_accuracy: 0.8906
Epoch 7/100
55000/55000 [===================

In [13]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 75us/sample - loss: 0.3147 - accuracy: 0.8934


In [14]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

OSError: SavedModel file does not exist at: my_model_A.h5/{saved_model.pbtxt|saved_model.pb}

In [15]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

NameError: name 'model_A' is not defined

In [16]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd",
                    metrics="accuracy")

NameError: name 'model_B_on_A' is not defined

In [17]:
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)

In [18]:
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)

In [19]:
def exponantial_decay(lr0, s):
    def exponantial_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponantial_decay_fn

In [22]:
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train, y_train, 
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb, lr_scheduler])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 9s 167us/sample - loss: 0.4226 - accuracy: 0.8468 - val_loss: 0.3783 - val_accuracy: 0.8646
Epoch 2/100
55000/55000 [==============================] - 9s 166us/sample - loss: 0.3574 - accuracy: 0.8680 - val_loss: 0.3666 - val_accuracy: 0.8664
Epoch 3/100
55000/55000 [==============================] - 9s 165us/sample - loss: 0.3254 - accuracy: 0.8802 - val_loss: 0.3650 - val_accuracy: 0.8668
Epoch 4/100
55000/55000 [==============================] - 9s 164us/sample - loss: 0.2999 - accuracy: 0.8881 - val_loss: 0.3345 - val_accuracy: 0.8806
Epoch 5/100
55000/55000 [==============================] - 9s 164us/sample - loss: 0.2801 - accuracy: 0.8963 - val_loss: 0.3141 - val_accuracy: 0.8860
Epoch 6/100
55000/55000 [==============================] - 9s 164us/sample - loss: 0.2572 - accuracy: 0.9029 - val_loss: 0.3233 - val_accuracy: 0.8830
Epoch 7/100
55000/55000 [====================

In [23]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 66us/sample - loss: 0.3156 - accuracy: 0.8952


In [24]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),    
    keras.layers.Dense(500, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [26]:
optimizer = keras.optimizers.Adam()
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=optimizer,
              metrics=["accuracy"])

In [27]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
history = model.fit(X_train, y_train, 
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb, lr_scheduler])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 10s 185us/sample - loss: 0.4804 - accuracy: 0.8268 - val_loss: 0.3757 - val_accuracy: 0.8618
Epoch 2/100
55000/55000 [==============================] - 9s 166us/sample - loss: 0.3726 - accuracy: 0.8635 - val_loss: 0.3282 - val_accuracy: 0.8742
Epoch 3/100
55000/55000 [==============================] - 9s 167us/sample - loss: 0.3369 - accuracy: 0.8746 - val_loss: 0.3156 - val_accuracy: 0.8832
Epoch 4/100
55000/55000 [==============================] - 9s 167us/sample - loss: 0.3044 - accuracy: 0.8866 - val_loss: 0.2999 - val_accuracy: 0.8902
Epoch 5/100
55000/55000 [==============================] - 9s 169us/sample - loss: 0.2848 - accuracy: 0.8928 - val_loss: 0.3039 - val_accuracy: 0.8930
Epoch 6/100
55000/55000 [==============================] - 9s 168us/sample - loss: 0.2639 - accuracy: 0.9005 - val_loss: 0.3109 - val_accuracy: 0.8874
Epoch 7/100
55000/55000 [===================

In [28]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 74us/sample - loss: 0.3196 - accuracy: 0.8933


In [29]:
s = 20 * len(X_train) // 32
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.Adam(learning_rate)

In [30]:
layer = keras.layers.Dense(100, activation="elu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01))

In [36]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                          activation="selu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(500),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax", 
                     kernel_initializer="glorot_uniform")
])

In [37]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [38]:
history = model.fit(X_train, y_train,
                   validation_data=(X_valid, y_valid),
                   epochs=100,
                   callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 5s 97us/sample - loss: 2.7632 - accuracy: 0.7746 - val_loss: 0.9901 - val_accuracy: 0.7462
Epoch 2/100
55000/55000 [==============================] - 5s 88us/sample - loss: 0.9107 - accuracy: 0.7903 - val_loss: 0.9019 - val_accuracy: 0.8008
Epoch 3/100
55000/55000 [==============================] - 5s 92us/sample - loss: 0.8866 - accuracy: 0.7999 - val_loss: 0.8826 - val_accuracy: 0.7972
Epoch 4/100
55000/55000 [==============================] - 5s 90us/sample - loss: 0.8804 - accuracy: 0.8049 - val_loss: 0.8477 - val_accuracy: 0.8158
Epoch 5/100
55000/55000 [==============================] - 5s 87us/sample - loss: 0.8742 - accuracy: 0.8058 - val_loss: 0.8265 - val_accuracy: 0.8230
Epoch 6/100
55000/55000 [==============================] - 5s 90us/sample - loss: 0.8661 - accuracy: 0.8108 - val_loss: 0.8548 - val_accuracy: 0.8224
Epoch 7/100
55000/55000 [==========================

In [39]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 44us/sample - loss: 0.8486 - accuracy: 0.8186


In [51]:

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(500, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"),
])

In [52]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [53]:
history = model.fit(X_train, y_train,
                   epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 7s 120us/sample - loss: 0.7010 - accuracy: 0.7488 - val_loss: 0.4460 - val_accuracy: 0.8370
Epoch 2/100
55000/55000 [==============================] - 6s 114us/sample - loss: 0.5265 - accuracy: 0.8053 - val_loss: 0.3940 - val_accuracy: 0.8536
Epoch 3/100
55000/55000 [==============================] - 6s 116us/sample - loss: 0.4942 - accuracy: 0.8174 - val_loss: 0.3810 - val_accuracy: 0.8554
Epoch 4/100
55000/55000 [==============================] - 6s 116us/sample - loss: 0.4761 - accuracy: 0.8245 - val_loss: 0.4213 - val_accuracy: 0.8396
Epoch 5/100
55000/55000 [==============================] - 6s 116us/sample - loss: 0.4584 - accuracy: 0.8323 - val_loss: 0.3574 - val_accuracy: 0.8706
Epoch 6/100
55000/55000 [==============================] - 6s 115us/sample - loss: 0.4431 - accuracy: 0.8380 - val_loss: 0.3660 - val_accuracy: 0.8706
Epoch 7/100
55000/55000 [====================

In [54]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 42us/sample - loss: 0.3377 - accuracy: 0.8772


In [65]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=15,
                                                 restore_best_weights=True)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(500, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.5),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.3),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"),
])

In [66]:
s = 20 * len(X_train) // 32 # number of steps in 20 epochs (batch size = 32)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

In [67]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.Adam(learning_rate=5e-4),
             metrics=["accuracy"])

In [68]:
history = model.fit(X_train, y_train,
                   epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 7s 125us/sample - loss: 0.7229 - accuracy: 0.7390 - val_loss: 0.4445 - val_accuracy: 0.8338
Epoch 2/100
55000/55000 [==============================] - 7s 119us/sample - loss: 0.5369 - accuracy: 0.8026 - val_loss: 0.4081 - val_accuracy: 0.8454
Epoch 3/100
55000/55000 [==============================] - 7s 119us/sample - loss: 0.4928 - accuracy: 0.8182 - val_loss: 0.3913 - val_accuracy: 0.8512
Epoch 4/100
55000/55000 [==============================] - 7s 120us/sample - loss: 0.4644 - accuracy: 0.8294 - val_loss: 0.3728 - val_accuracy: 0.8564
Epoch 5/100
55000/55000 [==============================] - 7s 118us/sample - loss: 0.4426 - accuracy: 0.8349 - val_loss: 0.3503 - val_accuracy: 0.8658
Epoch 6/100
55000/55000 [==============================] - 7s 118us/sample - loss: 0.4309 - accuracy: 0.8390 - val_loss: 0.3433 - val_accuracy: 0.8704
Epoch 7/100
55000/55000 [====================

55000/55000 [==============================] - 7s 119us/sample - loss: 0.3038 - accuracy: 0.8873 - val_loss: 0.2857 - val_accuracy: 0.8980
Epoch 55/100
55000/55000 [==============================] - 6s 116us/sample - loss: 0.2971 - accuracy: 0.8882 - val_loss: 0.2750 - val_accuracy: 0.8998
Epoch 56/100
55000/55000 [==============================] - 6s 118us/sample - loss: 0.2976 - accuracy: 0.8875 - val_loss: 0.2878 - val_accuracy: 0.8974
Epoch 57/100
55000/55000 [==============================] - 7s 119us/sample - loss: 0.2966 - accuracy: 0.8904 - val_loss: 0.2797 - val_accuracy: 0.8970
Epoch 58/100
55000/55000 [==============================] - 6s 117us/sample - loss: 0.2999 - accuracy: 0.8876 - val_loss: 0.2810 - val_accuracy: 0.8986
Epoch 59/100
55000/55000 [==============================] - 7s 119us/sample - loss: 0.2974 - accuracy: 0.8886 - val_loss: 0.2822 - val_accuracy: 0.8966
Epoch 60/100
55000/55000 [==============================] - 6s 117us/sample - loss: 0.2953 - accuracy

In [69]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 40us/sample - loss: 0.3053 - accuracy: 0.8909


In [91]:
y_probas = np.stack([model(X_test, training=True)
                       for sample in range(100)])

y_proba = y_probas.mean(axis=0)

In [92]:
np.round(model.predict(X_test[:1]), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],
      dtype=float32)

In [93]:
np.round(y_probas[:, :1], 2)

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.22, 0.  , 0.22, 0.  , 0.56]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.08, 0.  , 0.07, 0.  , 0.85]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.12, 0.  , 0.86]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.51, 0.  , 0.46]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.13, 0.  , 0.86]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.07, 0.  , 0.93]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.01, 0.  , 0.98]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.09, 0.  , 0.42, 0.  , 0.49]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.04, 0.  , 0

In [94]:
np.round(y_proba[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.04, 0.  , 0.1 , 0.  , 0.87]],
      dtype=float32)

In [95]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.1 , 0.  , 0.15, 0.  , 0.2 ]],
      dtype=float32)

In [96]:
y_pred = np.argmax(y_proba, axis=1)
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.8913

In [90]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [97]:
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                  kernel_constraint=keras.constraints.max_norm(1.))

In [98]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(500, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"),
])

In [99]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.Nadam(),
             metrics=["accuracy"])

In [100]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

In [101]:
history = model.fit(X_train, y_train,
                   epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb, lr_scheduler])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 10s 184us/sample - loss: 0.7377 - accuracy: 0.7446 - val_loss: 0.4297 - val_accuracy: 0.8410
Epoch 2/100
55000/55000 [==============================] - 10s 174us/sample - loss: 0.5443 - accuracy: 0.8024 - val_loss: 0.4059 - val_accuracy: 0.8524
Epoch 3/100
55000/55000 [==============================] - 10s 173us/sample - loss: 0.5020 - accuracy: 0.8166 - val_loss: 0.3975 - val_accuracy: 0.8566
Epoch 4/100
55000/55000 [==============================] - 10s 175us/sample - loss: 0.4862 - accuracy: 0.8203 - val_loss: 0.3744 - val_accuracy: 0.8634
Epoch 5/100
55000/55000 [==============================] - 10s 174us/sample - loss: 0.4723 - accuracy: 0.8281 - val_loss: 0.3843 - val_accuracy: 0.8598
Epoch 6/100
55000/55000 [==============================] - 9s 172us/sample - loss: 0.4678 - accuracy: 0.8305 - val_loss: 0.3541 - val_accuracy: 0.8674
Epoch 7/100
55000/55000 [===============

55000/55000 [==============================] - 10s 174us/sample - loss: 0.2947 - accuracy: 0.8915 - val_loss: 0.2737 - val_accuracy: 0.9002
Epoch 55/100
55000/55000 [==============================] - 10s 173us/sample - loss: 0.2961 - accuracy: 0.8903 - val_loss: 0.2748 - val_accuracy: 0.9004
Epoch 56/100
55000/55000 [==============================] - 9s 172us/sample - loss: 0.2918 - accuracy: 0.8907 - val_loss: 0.2744 - val_accuracy: 0.8992
Epoch 57/100
55000/55000 [==============================] - 9s 172us/sample - loss: 0.2900 - accuracy: 0.8918 - val_loss: 0.2742 - val_accuracy: 0.9008
Epoch 58/100
55000/55000 [==============================] - 10s 174us/sample - loss: 0.2900 - accuracy: 0.8913 - val_loss: 0.2707 - val_accuracy: 0.9020
Epoch 59/100
55000/55000 [==============================] - 10s 175us/sample - loss: 0.2867 - accuracy: 0.8931 - val_loss: 0.2697 - val_accuracy: 0.9036
Epoch 60/100
55000/55000 [==============================] - 10s 174us/sample - loss: 0.2875 - acc

In [102]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 43us/sample - loss: 0.3062 - accuracy: 0.8903


In [108]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.1),
    keras.layers.Dense(500, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.3),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.1),
    keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"),
])

In [109]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.Nadam(),
             metrics=["accuracy"])

In [110]:
history = model.fit(X_train, y_train,
                   epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb, lr_scheduler])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 12s 215us/sample - loss: 0.8760 - accuracy: 0.6718 - val_loss: 0.8886 - val_accuracy: 0.7148
Epoch 2/100
55000/55000 [==============================] - 11s 205us/sample - loss: 0.7192 - accuracy: 0.7303 - val_loss: 0.7470 - val_accuracy: 0.7904
Epoch 3/100
55000/55000 [==============================] - 11s 204us/sample - loss: 0.6836 - accuracy: 0.7429 - val_loss: 0.7350 - val_accuracy: 0.8030
Epoch 4/100
55000/55000 [==============================] - 11s 204us/sample - loss: 0.6626 - accuracy: 0.7501 - val_loss: 0.6259 - val_accuracy: 0.8064
Epoch 5/100
55000/55000 [==============================] - 11s 204us/sample - loss: 0.6470 - accuracy: 0.7581 - val_loss: 0.6751 - val_accuracy: 0.8018
Epoch 6/100
55000/55000 [==============================] - 11s 203us/sample - loss: 0.6354 - accuracy: 0.7614 - val_loss: 0.5958 - val_accuracy: 0.8154
Epoch 7/100
55000/55000 [==============

55000/55000 [==============================] - 11s 203us/sample - loss: 0.5137 - accuracy: 0.8050 - val_loss: 0.5174 - val_accuracy: 0.8572
Epoch 55/100
55000/55000 [==============================] - 11s 203us/sample - loss: 0.5074 - accuracy: 0.8080 - val_loss: 0.4747 - val_accuracy: 0.8620
Epoch 56/100
55000/55000 [==============================] - 11s 204us/sample - loss: 0.5050 - accuracy: 0.8091 - val_loss: 0.4764 - val_accuracy: 0.8616
Epoch 57/100
55000/55000 [==============================] - 11s 204us/sample - loss: 0.5025 - accuracy: 0.8083 - val_loss: 0.4450 - val_accuracy: 0.8692
Epoch 58/100
55000/55000 [==============================] - 11s 205us/sample - loss: 0.5084 - accuracy: 0.8072 - val_loss: 0.4644 - val_accuracy: 0.8650
Epoch 59/100
55000/55000 [==============================] - 11s 205us/sample - loss: 0.5024 - accuracy: 0.8097 - val_loss: 0.4727 - val_accuracy: 0.8642
Epoch 60/100
55000/55000 [==============================] - 11s 206us/sample - loss: 0.5000 - a

In [111]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 45us/sample - loss: 0.5186 - accuracy: 0.8539
